In [3]:
import torch
from torch import nn
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity
import cProfile

In [4]:
x = torch.rand(256, 512)
linear = nn.Linear(512, 256).eval()

with torch.inference_mode():
    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        linear(x)
print(prof.key_averages().table())

----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
          aten::linear         0.51%      33.000us       100.00%       6.531ms       6.531ms             1  
               aten::t        10.37%     677.000us        20.10%       1.313ms       1.313ms             1  
       aten::transpose         0.38%      25.000us         9.74%     636.000us     636.000us             1  
      aten::as_strided         9.37%     612.000us         9.37%     612.000us     306.000us             2  
           aten::addmm        66.24%       4.326ms        79.39%       5.185ms       5.185ms             1  
          aten::expand         0.17%      11.000us         0.18%      12.000us      12.000us             1  
           aten::co

STAGE:2023-11-22 14:02:57 143217:143217 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-11-22 14:02:57 143217:143217 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-11-22 14:02:57 143217:143217 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [5]:
jit = torch.jit.script(linear)
torch.jit.save(jit, 'linear.pt')

In [3]:
from torch.ao.quantization import get_default_qconfig
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx
from torch.ao.quantization import QConfigMapping

with torch.inference_mode():
    torch.backends.quantized.engine = 'x86'
    qconfig = get_default_qconfig('x86')
    qconfig_mapping = QConfigMapping().set_global(qconfig)
    example_inputs = torch.randn_like(x)
    prepared_model = prepare_fx(linear, qconfig_mapping, example_inputs)
    for _ in range(16):
        prepared_model(torch.randn_like(x))

    quantized_model = convert_fx(prepared_model)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        quantized_model(x)

print(prof.key_averages().table())
print(quantized_model.code)

/home/alexey/.local/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/alexey/.local/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/alexey/.local/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/alexey/.local/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
/home/alexey/.local/lib/python3.10/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        aten::quantize_per_tensor        14.80%     776.000us        14.98%     785.000us     785.000us             1  
                       aten::item         0.25%      13.000us         0.31%      16.000us       4.000us             4  
        aten::_local_scalar_dense         0.08%       4.000us         0.08%       4.000us       1.000us             4  
                quantized::linear        83.33%       4.368ms        83.88%       4.397ms       4.397ms             1  
    aten::_empty_affine_quantized         0.29%      15.000us         0.29%      15.000us      15.000us             1  
                    aten::q_scale       

/home/alexey/.local/lib/python3.10/site-packages/torch/fx/graph.py:1377: UserWarning: Node _packed_weight_0 target _packed_weight_0 _packed_weight_0 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
STAGE:2023-11-20 19:58:42 112085:112085 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-11-20 19:58:42 112085:112085 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-11-20 19:58:42 112085:112085 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [8]:
from torch.ao.quantization import get_default_qconfig
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx
from torch.ao.quantization import QConfigMapping

with torch.inference_mode():
    torch.backends.quantized.engine = 'fbgemm'
    qconfig = get_default_qconfig('fbgemm')
    qconfig_mapping = QConfigMapping().set_global(qconfig)
    example_inputs = torch.randn_like(x)
    prepared_model = prepare_fx(linear, qconfig_mapping, example_inputs)
    for _ in range(16):
        prepared_model(torch.randn_like(x))

    quantized_model = convert_fx(prepared_model)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        quantized_model(x)

print(prof.key_averages().table())
print(quantized_model.code)

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        aten::quantize_per_tensor        14.06%      70.000us        15.26%      76.000us      76.000us             1  
                       aten::item         1.81%       9.000us         2.01%      10.000us       2.500us             4  
        aten::_local_scalar_dense         0.20%       1.000us         0.20%       1.000us       0.250us             4  
                quantized::linear        77.51%     386.000us        79.92%     398.000us     398.000us             1  
    aten::_empty_affine_quantized         1.00%       5.000us         1.00%       5.000us       5.000us             1  
                    aten::q_scale       

STAGE:2023-11-20 19:59:44 112085:112085 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-11-20 19:59:44 112085:112085 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-11-20 19:59:44 112085:112085 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [4]:
from torch.ao.quantization import get_default_qconfig
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx
from torch.ao.quantization import QConfigMapping

with torch.inference_mode():

    qconfig = get_default_qconfig('fbgemm')
    qconfig_mapping = QConfigMapping().set_global(qconfig)
    example_inputs = torch.randn_like(x)
    prepared_model = prepare_fx(linear, qconfig_mapping, example_inputs)
    for _ in range(16):
        prepared_model(torch.randn_like(x))

    quantized_model = convert_fx(prepared_model)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        quantized_model(x)

print(prof.key_averages().table())
print(quantized_model.code)

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        aten::quantize_per_tensor        85.14%       3.369ms        85.39%       3.379ms       3.379ms             1  
                       aten::item         0.38%      15.000us         0.51%      20.000us       5.000us             4  
        aten::_local_scalar_dense         0.15%       6.000us         0.15%       6.000us       1.500us             4  
                quantized::linear        12.56%     497.000us        13.19%     522.000us     522.000us             1  
    aten::_empty_affine_quantized         0.28%      11.000us         0.28%      11.000us      11.000us             1  
                    aten::q_scale       

STAGE:2023-11-20 19:58:42 112085:112085 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-11-20 19:58:42 112085:112085 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-11-20 19:58:42 112085:112085 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [5]:
jit = torch.jit.script(quantized_model)
# torch.jit.save(jit, 'fbgemm_linear.pt')

/home/alexey/.local/lib/python3.10/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(


In [6]:
print(jit.graph)

graph(%self : __torch__.torch.fx.graph_module.GraphModule,
      %input.1 : Tensor):
  %8 : int = prim::Constant[value=13]() # <eval_with_key>.15:8:98
  %_input_scale_0.1 : Tensor = prim::GetAttr[name="_input_scale_0"](%self)
  %_input_zero_point_0.1 : Tensor = prim::GetAttr[name="_input_zero_point_0"](%self)
  %quantize_per_tensor.1 : Tensor = aten::quantize_per_tensor(%input.1, %_input_scale_0.1, %_input_zero_point_0.1, %8) # <eval_with_key>.15:8:26
  %_packed_weight_0.1 : __torch__.torch.classes.quantized.LinearPackedParamsBase = prim::GetAttr[name="_packed_weight_0"](%self)
  %_scale_1.1 : Tensor = prim::GetAttr[name="_scale_1"](%self)
  %_zero_point_1.1 : Tensor = prim::GetAttr[name="_zero_point_1"](%self)
  %21 : float = aten::FloatImplicit(%_scale_1.1) # <eval_with_key>.15:12:13
  %22 : int = aten::IntImplicit(%_zero_point_1.1) # <eval_with_key>.15:12:13
  %linear.1 : Tensor = quantized::linear(%quantize_per_tensor.1, %_packed_weight_0.1, %21, %22) # <eval_with_key>.15:12:13
  %

In [7]:
from torch.ao.quantization import get_default_qconfig
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx
from torch.ao.quantization import QConfigMapping

conv2d = nn.Conv2d(16, 8, 3)
image = torch.randn(1, 16, 64, 64)

with torch.inference_mode():

    qconfig = get_default_qconfig('fbgemm')
    qconfig_mapping = QConfigMapping().set_global(qconfig)
    example_inputs = torch.randn_like(image)
    prepared_model = prepare_fx(conv2d, qconfig_mapping, example_inputs)
    for _ in range(16):
        prepared_model(torch.randn_like(image))

    quantized_model = convert_fx(prepared_model)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        quantized_model(image)

print(prof.key_averages().table())
print(quantized_model.code)

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        aten::quantize_per_tensor         0.23%      60.000us         0.26%      68.000us      68.000us             1  
                       aten::item         0.03%       9.000us         0.04%      11.000us       2.750us             4  
        aten::_local_scalar_dense         0.01%       2.000us         0.01%       2.000us       0.500us             4  
                quantized::conv2d        35.61%       9.285ms        36.00%       9.387ms       9.387ms             1  
                 aten::contiguous         0.01%       2.000us         0.36%      95.000us      95.000us             1  
                      aten::clone       

STAGE:2023-11-20 19:58:43 112085:112085 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-11-20 19:58:43 112085:112085 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-11-20 19:58:43 112085:112085 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
